In [ ]:
#|default_exp filter

In [ ]:
#|export
from __future__ import annotations
import json,shutil

from nbprocess.read import *
from nbprocess.sync import *
from nbprocess.process import *
from nbprocess.processors import *
from fastcore.utils import *
from fastcore.script import call_parse,bool_arg

# quarto

> Filter for quarto processing

In [ ]:
#|export
class FilterDefaults:
    def _nothing(self): return []
    xtra_procs=xtra_preprocs=xtra_postprocs=_nothing
    
    def base_procs(self):
        return [strip_ansi, meta_, hide_line, filter_stream_, clean_magics,
                bash_identify, rm_header_dash, rm_export, clean_show_doc, remove_]
    def base_preprocs(self): return [insert_warning]
    def base_postprocs(self): return []

    def procs(self):
        "Processors for export"
        return self.base_procs() + self.xtra_procs()

    def preprocs(self):
        "Preprocessors for export"
        return self.base_preprocs() + self.xtra_preprocs()

    def postprocs(self):
        "Postprocessors for export"
        return self.base_postprocs() + self.xtra_postprocs()

In [ ]:
#|export
@call_parse
def filter_nb(
    nb_txt:str=None  # Notebook text (uses stdin if not provided)
):
    filt = get_config().get('exporter', FilterDefaults)()
    printit = False
    if not nb_txt:
        nb_txt = sys.stdin.read()
        printit = True
    nb = dict2nb(json.loads(nb_txt))
    nbp = NBProcessor(nb=nb, procs=filt.procs(), preprocs=filt.preprocs(), postprocs=filt.postprocs())
    nbp.process()
    res = nb2str(nb)
    if printit: print(res)
    else: return res

In [ ]:
#|export
@call_parse
def create_quarto(
    path:str=None, # path to notebooks
    doc_path:str=None, # path to output docs
    symlinks:bool=False, # follow symlinks?
    file_glob:str='*.ipynb', # Only include files matching glob
    file_re:str=None, # Only include files matching regex
    folder_re:str=None, # Only enter folders matching regex
    skip_file_glob:str=None, # Skip files matching glob
    skip_file_re:str=None, # Skip files matching regex
    skip_folder_re:str='^[_.]' # Skip folders matching regex
):
    cfg = get_config()
    path = cfg.path("nbs_path") if not path else Path(path)
    doc_path = cfg.path("doc_path") if not doc_path else Path(doc_path)
    files = globtastic(path, symlinks=symlinks, file_glob=file_glob, file_re=file_re,
                       folder_re=folder_re, skip_file_glob=skip_file_glob,
                       skip_file_re=skip_file_re, skip_folder_re=skip_folder_re
                      ).sorted().map(Path)
    yml_path = path/'_quarto.yml'
    yml = L(yml_path.read_text().splitlines())
    re_contents = re.compile('^\s+contents:\s*$').search
    loc = yml.argfirst(re_contents)
    prefix = re.findall('^(\s+)contents:', yml[loc])[0]
    yml = yml[:loc+1]
    yml += [f'  {prefix}- {o.relative_to(path)}' for o in files]
    yml_path.write_text('\n'.join(yml))
    os.system(f'cd {path} && quarto render')
    os.system(f'cd {path} && quarto render {files[0]} -o README.md -t gfm')
    cfg_path = cfg.config_path
    shutil.rmtree(cfg_path/'docs', ignore_errors=True)
    (cfg_path/'README.md').unlink(missing_ok=True)
    site = path/'_site'
    shutil.move(site/'README.md', cfg_path)
    shutil.move(site, cfg_path/'docs')

In [ ]:
create_quarto()

[ 1/10] 09_filter.ipynb
[ 2/10] 08_processors.ipynb
[ 3/10] 06_merge.ipynb
[ 4/10] 03_process.ipynb
[ 5/10] 04_export.ipynb
[ 6/10] 02_maker.ipynb
[ 7/10] 07_clean.ipynb
[ 8/10] 00_index.ipynb
[ 9/10] 01_read.ipynb
[10/10] 05_sync.ipynb

Output created: _site/09_filter.html

pandoc -o README.md
  to: gfm+footnotes+tex_math_dollars-yaml_metadata_block
  standalone: true
  default-image-extension: png
  html-math-method: webtex
  filters:
    - crossref
  
Output created: _site/README.md



## Export -

In [ ]:
#skip
from nbprocess.export import nbs_export
nbs_export()